In [ ]:
%%capture

!pip install gradio
!pip install torch
!pip install unsloth
!pip install bitsandbytes

In [ ]:
import gradio as gr
import torch
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template


lora_adapter = "Braszczynski/Llama-3.2-3B-Instruct-bnb-4bit-merged-v2-1E"

max_seq_length = 512
dtype = None
load_in_4bit = True   # Set to True if you want to use 4-bit quantization

# Load Model and Tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = lora_adapter,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Apply Chat Template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

# Enable Faster Inference and Move Model to Device
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

def respond(message, system_message, max_tokens, temperature, top_p):
    # Combine system message and user message to form the chat history
    chat_history = f"{system_message}\nUser: {message}\nAssistant:"

    # Prepare the input for the model
    inputs = tokenizer(
        chat_history,
        return_tensors="pt",
        truncation=True,
        max_length=max_seq_length,
    ).to(device)

    # Generate the response
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            max_new_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            pad_token_id=tokenizer.eos_token_id,
            use_cache=True
        )

    # Decode and format the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response = response[len(chat_history):].strip()  # Remove the input context
    response = response.split("User:")[0].strip()  # Remove any trailing user messages
    response = response.split("\n")[0].strip()  # Additional cleanup if needed
    return response

# Define System Message
system_message = (
    "You are a fact checker/proofreader. You should reply exactly with a message from the user, "
    "but you should fix all the mistakes that it contains, and then list the mistakes at the bottom. "
    "Example user message would be \"Titanic was a plane\" and you should reply with \"Titanic was a ship\". "
    "Nothing more don't continue the conversation and NO explanation!"
)

# Define Gradio Interface
demo = gr.Interface(
    fn=respond,
    inputs=[
        gr.TextArea(label="User Message", placeholder="Enter your message here...", lines=5),
        gr.Textbox(value=system_message, label="System Message"),
        gr.Slider(minimum=1, maximum=512, value=120, step=1, label="Max New Tokens"),
        gr.Slider(minimum=0.1, maximum=4.0, value=0.1, step=0.1, label="Temperature"),
        gr.Slider(minimum=0.1, maximum=1.0, value=0.2, step=0.05, label="Top-p (Nucleus Sampling)"),
    ],
    outputs=gr.TextArea(
        label="Output",
        interactive=False,
        show_copy_button=True,
    ),
    flagging_mode="never",
    title="Fact Checker",
    description="Enter a message, and the system will correct any mistakes and list them.",
    allow_flagging="never",

)

if __name__ == "__main__":
    demo.launch()


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/gradio/interface.py:399: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://144f06d1f46397faf1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
